In [3]:
import sys
!{sys.executable} -m pip -q install pandas
!{sys.executable} -m pip -q install geopandas
!{sys.executable} -m pip -q install folium
!{sys.executable} -m pip -q install ipyleaflet
!{sys.executable} -m pip -q install matplotlib
!{sys.executable} -m pip -q install geopy



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Read in CITI Bike Station Locations

In [1]:
import json
bikeStationsFilepath = "./input/citi_bike_station_information.json"
fp = open(bikeStationsFilepath, "r")
stationInformation = json.load(fp)
bikeStations = stationInformation["data"]["stations"]

# NYC Income Data

In [8]:
import geopandas as gpd
from shapely import Point

In [9]:
boroughCol = "borough"
neighborhoodsGeometryFilepath = "./input/nyc_neighborhoods.geojson"
dfNeighborhoods = gpd.read_file(neighborhoodsGeometryFilepath)
manhattanFilter = dfNeighborhoods[boroughCol] == "Manhattan"
dfManhattan = dfNeighborhoods[manhattanFilter]

Differing neighborhood granularity amongst income data and GeoJSON data. Normalize the neighborhoods with manual mappings?

# Subway Stops

In [10]:
import pandas as pd
from geopy.distance import distance
from geopy.geocoders import Nominatim
import numpy as np
import time

In [ ]:
trainStationsFilepath = "./input/subway_stops.txt"
trainStopIdCol = "stop_id"
trainStopLatCol = "stop_lat"
trainStopLongCol = "stop_lon"
bikeStationIdCol = "station_id"
bikeStationNameCol = "name"
bikeStationLatitudeCol = "lat"
bikeStationLongitudeCol = "lon"
addressField = "address"
neighborhoodField = "neighbourhood"
boroughField = "borough"
trainStopsAugmentedFilepath = "./input/subway_stops_augmented.csv"
bikeStationsAugmentedFilepath = "./input/bike_stations_augmented.csv"
dfTrainStops = pd.read_csv(trainStationsFilepath)
dfBikeStations = pd.DataFrame(columns = [bikeStationIdCol, bikeStationNameCol, bikeStationLatitudeCol, bikeStationLongitudeCol])
#Remove duplicate stops
dfTrainStopsFiltered = dfTrainStops[~((dfTrainStops[trainStopIdCol].str.endswith("N")) | (dfTrainStops[trainStopIdCol].str.endswith("S")))]

Geocode train station data

In [12]:
def reverse_geocode_coordinates(latitude, longitude):
    neighborhoodGeojsonField = "neighborhood"
    geometryField = "geometry"
    neighborhood = "None"
    borough = "None"
    for index, row in dfNeighborhoods.iterrows():
        polygon = row[geometryField]
        point = Point(longitude, latitude)
        if point.within(polygon):
           neighborhood = row[neighborhoodGeojsonField]
           borough = row[boroughField]
           break
    return {neighborhoodField: neighborhood, boroughCol: borough}

In [13]:
boroughVals = list()
neighborhoodVals = list()
for index, row in dfTrainStopsFiltered.iterrows():
    #Get location data and extract neighborhood and borough for the provided coordinates
    locationData = reverse_geocode_coordinates(row[trainStopLatCol], row[trainStopLongCol])
    neighborhoodVals.append(locationData[neighborhoodField])
    boroughVals.append(locationData[boroughField])
dfTrainStopsFiltered[neighborhoodField] = neighborhoodVals
dfTrainStopsFiltered[boroughField] = boroughVals
#Save reverse geocoded data to file
dfTrainStopsFiltered.to_csv(trainStopsAugmentedFilepath, index = False)

/var/folders/33/0fb406ns7l1dyhs68vq615540000gp/T/ipykernel_4139/418968998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrainStopsFiltered[neighborhoodField] = neighborhoodVals
/var/folders/33/0fb406ns7l1dyhs68vq615540000gp/T/ipykernel_4139/418968998.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrainStopsFiltered[boroughField] = boroughVals


Augment Citi bike stations with neighborhood and borough information

In [14]:
#Extract latitude and longitude for bike stations and store in dataframe
for stationObj in bikeStations:
    rowVals = [stationObj[bikeStationIdCol], stationObj[bikeStationNameCol], stationObj[bikeStationLatitudeCol], stationObj[bikeStationLongitudeCol]]
    dfBikeStations.loc[len(dfBikeStations)] = rowVals

In [15]:
boroughVals = list()
neighborhoodVals = list()
for index, row in dfBikeStations.iterrows():
    #Get location data and extract neighborhood and borough for the provided coordinates
    locationData = reverse_geocode_coordinates(row[bikeStationLatitudeCol], row[bikeStationLongitudeCol])
    neighborhoodVals.append(locationData[neighborhoodField])
    boroughVals.append(locationData[boroughField])
dfBikeStations[neighborhoodField] = neighborhoodVals
dfBikeStations[boroughField] = boroughVals
#Save reverse geocoded data to file
dfBikeStations.to_csv(bikeStationsAugmentedFilepath, index = False)

Compute distance to closest bike station for each train station

In [158]:
def closest_bike_station_distance(trainStopCoord, dfBikeStations):
    #Initialize variable to track the distance from the train stop to the closest bike station
    closestBikeStationDistance = np.inf
    #Find the closest bike station out of all the possible bike stations
    for index, row in dfBikeStations.iterrows():
        bikeStationCoord = (row[bikeStationLatitudeCol], row[bikeStationLongitudeCol])
        #Compute distance between train stop and bike station using GeoPy package
        bikeStationDistance = distance(trainStopCoord, bikeStationCoord).km
        if bikeStationDistance < closestBikeStationDistance:
            closestBikeStationDistance = bikeStationDistance
    return closestBikeStationDistance

In [159]:
#For each train stop, compute the distance and append this to dataframe for train stops
distanceCol = "distance_to_closest_bike_station"
dfTrainStopsAugmented = pd.read_csv(trainStopsAugmentedFilepath)
dfBikeStationsAugmented = pd.read_csv(bikeStationsAugmentedFilepath)
distanceVals = list()
for index, row in dfTrainStopsAugmented.iterrows():
    trainStopCoord = (row[trainStopLatCol], row[trainStopLongCol])
    distanceVals.append(closest_bike_station_distance(trainStopCoord, dfBikeStations))
dfTrainStopsAugmented[distanceCol] = distanceVals
dfTrainStopsAugmented.to_csv(trainStopsAugmentedFilepath, index = False)

Instead of making a table grouped by subway line, make a table grouped by borough or neighborhood (use lat/long of station to geocode)

Number of train stations, number of Citi bike stations, average distance from train station to closest bike station

In [35]:
import pandas as pd
distanceCol = "distance_to_closest_bike_station"
neighborhoodStatsFilepath = "./output/manhattan_neighborhood_stats.csv"
boroughStatsFilepath = "./output/borough_stats.csv"

In [36]:
dfBikeStationsAugmented = pd.read_csv(bikeStationsAugmentedFilepath)
manhattanFilter = dfBikeStationsAugmented[boroughCol] == "Manhattan"
dfBikeStationsAugmented = dfBikeStationsAugmented[manhattanFilter]
dfBikeStationsGrouped = dfBikeStationsAugmented.groupby(neighborhoodField).agg({bikeStationIdCol:"count"}).reset_index()
dfBikeStationsGrouped = dfBikeStationsGrouped.rename(columns = {"station_id":"bike_station_count"})
dfTrainStopsAugmented = pd.read_csv(trainStopsAugmentedFilepath)
dfTrainStopsGrouped = dfTrainStopsAugmented.groupby(neighborhoodField).agg({trainStopIdCol:"count", distanceCol:"mean"}).reset_index()
dfTrainStopsGrouped = dfTrainStopsGrouped.rename(columns = {"stop_id":"train_station_count", "distance_to_closest_bike_station":"average_distance_to_bike_station"})
dfTrainStopsGrouped["average_distance_to_bike_station"] = round(dfTrainStopsGrouped["average_distance_to_bike_station"], 3)
dfResult = dfTrainStopsGrouped.merge(dfBikeStationsGrouped, left_on = neighborhoodField, right_on = neighborhoodField, how = "inner")
dfResult = dfResult.loc[:, ["neighbourhood", "train_station_count", "bike_station_count", "average_distance_to_bike_station"]]
dfResult = dfResult.rename(columns = {"neighbourhood":"Neighborhood", "train_station_count":"# Train Stations", "bike_station_count":"# Bike Stations", "average_distance_to_bike_station":"Mean Distance from Train Stop to Bike Station (km)"})
dfResult.to_csv(neighborhoodStatsFilepath, index = False)

In [37]:
dfBikeStationsAugmented = pd.read_csv(bikeStationsAugmentedFilepath)
dfBikeStationsGrouped = dfBikeStationsAugmented.groupby(boroughField).agg({bikeStationIdCol:"count"}).reset_index()
dfBikeStationsGrouped = dfBikeStationsGrouped.rename(columns = {"station_id":"bike_station_count"})
dfTrainStopsAugmented = pd.read_csv(trainStopsAugmentedFilepath)
dfTrainStopsGrouped = dfTrainStopsAugmented.groupby(boroughField).agg({trainStopIdCol:"count", distanceCol:"mean"}).reset_index()
dfTrainStopsGrouped = dfTrainStopsGrouped.rename(columns = {"stop_id":"train_station_count", "distance_to_closest_bike_station":"average_distance_to_bike_station"})
dfTrainStopsGrouped["average_distance_to_bike_station"] = round(dfTrainStopsGrouped["average_distance_to_bike_station"], 3)
dfResult = dfTrainStopsGrouped.merge(dfBikeStationsGrouped, left_on = boroughField, right_on = boroughField, how = "inner")
dfResult = dfResult.loc[:, ["borough", "train_station_count", "bike_station_count", "average_distance_to_bike_station"]]
dfResult = dfResult.rename(columns = {"borough":"Borough", "train_station_count":"# Train Stations", "bike_station_count":"# Bike Stations", "average_distance_to_bike_station":"Mean Distance from Train Stop to Bike Station (km)"})
dfResult.to_csv(boroughStatsFilepath, index = False)

Income Plot

In [123]:
mappingsFilepath = "./input/neighborhood_mappings.csv"
incomeFilepath = "./input/nyc_neighborhood_incomes.csv"
dfMappings = pd.read_csv(mappingsFilepath)

In [ ]:
#dfMappings = dfMappings.rename(columns = {"source_neighborhood": "neighborhood"})
dfManhattanNormalized = dfManhattan.merge(dfMappings, left_on = "neighborhood", right_on = "source_neighborhood", how = "inner")
dfManhattanNormalized = dfManhattanNormalized[["geometry", "source_neighborhood", "standardized_neighborhood"]]
# Drop the 'location' column (optional to do before or after)
dfManhattanNormalized = dfManhattanNormalized.drop(columns=["source_neighborhood"])
# Merge polygons by 'grouped_location'
merged = dfManhattanNormalized.dissolve(by="standardized_neighborhood")
dfIncomes = pd.read_csv(incomeFilepath)
dfIncomes = dfIncomes.rename(columns = {"Location":"standardized_neighborhood", "Data":"Annual Income"})
dfIncomes = dfIncomes[["standardized_neighborhood", "Annual Income"]]
dfManhattanIncomes = merged.merge(dfIncomes, left_on = "standardized_neighborhood", right_on = "standardized_neighborhood", how = "inner")

In [ ]:
from branca.colormap import linear
from folium.plugins import MarkerCluster
manhattanFilter = dfBikeStationsAugmented[boroughCol] == "Manhattan"
dfManhattanStations = dfBikeStationsAugmented[manhattanFilter]

# Create the base map
m = folium.Map(
    location=[40.7738, -73.9660],
    zoom_start=14,
    tiles='CartoDB positron',
)

# Define colormap for Classic Bike Rides
colormap = linear.YlGn_09.scale(
    int(dfManhattanIncomes['Annual Income'].min()), int(dfManhattanIncomes['Annual Income'].max())
)

# Adding GeoJsonTooltip to display both classic and electric bike data when both layers are shown
tooltip_combined = folium.GeoJsonTooltip(
    fields=['standardized_neighborhood', 'Annual Income'],  # Include both fields
    aliases=['Neighborhood:', 'Annual Income:'],  # Add appropriate aliases
    localize=True,
    sticky=True
)

folium.GeoJson(
    dfManhattanIncomes,  # This is your GeoDataFrame or GeoJSON data
    style_function=lambda feature: {
        "fillColor": colormap(feature["properties"]["Annual Income"]),
        "color": "transparent",  # Set border color to transparent
        "weight": 0,  # No visible border
        "fillOpacity": 0.7,  # Set fill opacity for the polygons
    },
    tooltip=tooltip_combined,
    popup=False
).add_to(m)

# Function to add circle markers
def add_markers(df, feature_group, color):
    for idx, row in df.iterrows():
        popup_content = f"""
            <b>Neighborhood:</b> {row['neighbourhood']}<br>
        """
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_content, max_width=300),
        ).add_to(feature_group)

markerCluster = MarkerCluster().add_to(m)

for index, row in dfManhattanStations.iterrows():
    folium.Marker(
        location = [row["lat"], row["lon"]],
        popup = row["borough"],
        icon = None
    ).add_to(markerCluster)

markerCluster.add_to(m)
# Show the map
m